In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
import pandas as pd
import glob
import warnings

def standardize_data(path = '/content/drive/Shareddrives/MATH456/FRED CSVs'):
  files = glob.glob(path + "/*.csv")

  complete_data = []

  #this loop combines all of the data together into a single csv file
  for single_file in files:
      df = pd.DataFrame(pd.read_csv(single_file))

      #retrieves name of the variable we want
      variable = list(df.columns)[1]
      df['DATE'] = pd.to_datetime(df['DATE'])

      #since not all of the data is the same, we had figured that standardizing
      #it to quarterly averages will be the best way to attack this issue
      df['quarter'] = df['DATE'].dt.to_period('Q')

      #this gets the name of the numeric dataset
      df = df[df[variable] != '.']

      #good data practice to convert calculation data into numerical values
      #as these were string data before
      df[variable] = pd.to_numeric(df[variable])
      
      #we only want data from the latest first date in each dataset to 
      #earliest end date in each dataset. will be changed if needed
      df = df.loc[df['DATE'].between('2012-10-31','2022-04-01', inclusive=True)]
      complete_data.append(df.groupby(['quarter'])[variable].mean().round(2))
  
  #renaming the columns in the data to make it easier for us to use
  for name in range(len(files)):
    files[name] = files[name].split('/',files[name].count('/'))
    files[name] = files[name][-1].split('.csv')[0].lstrip()

  complete_set = pd.concat(complete_data, axis=1).fillna(0)
  complete_set.columns = files

  #wrote to new file outside of where the raw datasets were
  #this is helpful in case we need to make changes to dates needed in future 
  complete_set.to_csv(path.strip('FRED CSVs') + 'Cleaned.csv')

  return "Done."

warnings.filterwarnings('ignore')
standardize_data()

'Done.'